In [1]:
import dill
from direct_graph import DirectedGraph
from direct_graph import topk
import sys
import matplotlib.pyplot as plt
import random
from datetime import datetime
import concurrent.futures
import time



def Run_LTM(graph, seeds, rounds, centrality):
	v, e = graph.size()
	print("* running LTM({}) TOP {} {} - graph size: {} {}".format(rounds, len(seeds), centrality, v, e))
	influenced, kept, steps = graph.ltm(seeds, rounds)
	print("** influenced(%d) kept(%d) steps(%d)" % (len(influenced), len(kept), steps))
	return len(influenced), len(kept), steps


In [2]:
random.seed(datetime.now())
seed = 100

lin_max_values = []
eigenc_max_values = []
bet_max_values = []

In [4]:
rounds = 10
NODES = 7115
min_edges = 75000
max_edges = 125000
incr = 0.001
p = random.uniform(0.001, 0.0028) # probability
seed = 100
graph = None
while p > 0:
    print("generating graph with N={} p={}".format(NODES, p))
    graph = DirectedGraph.randomDirectedGraph(NODES, p)
    edges = graph.size()[1]
    print("prob={} edges={}".format(p, edges))
    if edges >= min_edges and edges <= max_edges:
        break
    elif edges > max_edges:
        p += ((max_edges - edges) / min_edges) * incr
    else:
        p += ((min_edges - edges) / min_edges) * incr
    sys.stdout.flush()

print('# Edges = %d\tAverage Clustering = %f' % (graph.countEdges(), graph.toUndirect().average_clustering()))
sys.stdout.flush()

generating graph with N=7115 p=0.002219695056465389
prob=0.002219695056465389 edges=112047
# Edges = 112047	Average Clustering = 0.009282


In [ ]:
print('# Eigenvector Centrality...')
diffsum, cscores = graph.eigenvector_centrality()
# print(diffsum)
# print(cscores)
top_eigenc = [a for a, b in topk(cscores, seed)]
print(top_eigenc)
print('# Done')
sys.stdout.flush()

In [ ]:
print('# Betweennes centrality...')
bet = graph.betweenness()
# print(bet)
top_bet = [a for a, b in topk(bet, seed)]
print(top_bet)
print('# Done')
sys.stdout.flush()

In [ ]:
print("# Lin's index...")
lin = graph.lin_index()
#print(lin)
top_lin = [a for a, b in topk(lin, seed)]
print(top_lin)
print('# Done')
sys.stdout.flush()

In [ ]:
max_lin_influenced = Run_LTM(graph, top_lin[:seed], rounds, 'Lin')[0]
max_eigenc_influenced = Run_LTM(graph, top_eigenc[:seed], rounds, 'Eigenvector')[0]
max_bet_influenced = Run_LTM(graph, top_bet[:seed], rounds, 'Betweenness')[0]
lin_max_seed = seed
eigenc_max_seed = seed
bet_max_seed = seed

In [ ]:
while seed > 0:
    seed -= 5
    influenced_lin = Run_LTM(graph, top_lin[:seed], rounds, 'Lin')[0]
    if max_lin_influenced <= influenced_lin:
        max_lin_influenced = influenced_lin
        lin_max_seed = seed
    else:
        break

In [ ]:
seed = 100
while seed > 0:
    seed -= 5
    influenced_eigenc = Run_LTM(graph, top_eigenc[:seed], rounds, 'Eigenvector')[0]
    if max_eigenc_influenced <= influenced_eigenc:
        max_eigenc_influenced = influenced_eigenc
        eigenc_max_seed = seed
    else:
        break

In [ ]:
seed = 100
while seed > 0:
    seed -= 5
    influenced_bet = Run_LTM(graph, top_bet[:seed], rounds, 'Betweenness')[0]
    if max_bet_influenced <= influenced_bet:
        max_bet_influenced = influenced_bet
        bet_max_seed = seed
    else:
        break

In [ ]:
%matplotlib inline

In [ ]:
fig, ax = plt.subplots()

bar_width = 0.35

opacity = 0.4

rects1 = plt.bar(1, max_lin_influenced, width=bar_width, alpha=opacity, color='b', label='Lin')

rects2 = plt.bar(2, max_eigenc_influenced, width=bar_width, alpha=opacity, color='r', label='Eigenvector')

rects3 = plt.bar(3, max_bet_influenced, width=bar_width, alpha=opacity, color='y', label='Betweenness')

plt.xlabel('Centrality Measures')
plt.ylabel('Influenced')
plt.title('Influenced Comparison')
plt.xticks([1.2,2.2,3.2], ('L', 'E', 'B'))

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.tight_layout()

plt.savefig('rd.png')
